In [99]:
import numpy as np
import pandas as pd
from utils import df_filter_times,Position,Heap
from matplotlib import pyplot as plt

In [100]:
df_all = pd.read_csv("./data/1m-10d.csv",parse_dates=["datetime"])
df_market = df_filter_times(df_all).copy()

In [1]:
df_market['date'] = df_market['datetime'].dt.date

NameError: name 'df_market' is not defined

In [116]:
# Group by stock and date to handle each stock's data separately for each day
grouped = df_market.groupby(['name', 'date'])

# Function to calculate the difference with the last minute's price of the day
def calculate_difference(group):
    last_price = group['close'].iloc[-5]
    group['price_diff'] = last_price - group['close']
    return group

# Apply the function to each group
df_with_diff = grouped.apply(calculate_difference, include_groups=False)
df_with_diff

open     high      low    close  volume  \
name date                                                             
A    2024-04-08 222111   143.800  144.360  143.010  143.600   18409   
                222112   143.985  143.985  143.390  143.720    2000   
                222113   143.720  144.540  143.720  144.540    2300   
                222114   144.900  145.200  144.570  145.150    9806   
                222115   144.805  145.790  144.805  145.570    1507   
...                          ...      ...      ...      ...     ...   
ZTS  2024-04-19 1108850  146.400  146.440  146.380  146.395   24725   
                1108851  146.390  146.400  146.220  146.395   58488   
                1108852  146.395  146.430  146.275  146.380   65654   
                1108853  146.380  146.580  146.220  146.490  123368   
                1108854  146.500  146.500  146.500  146.500  480428   

                                         datetime  price_diff  
name date                                                      
A    2024-04-08 222111  2024-04-08 09:30:00-04:00       0.960  
                222112  2024-04-08 09:31:00-04:00       0.840  
                222113  2024-04-08 09:32:00-04:00       0.020  
                222114  2024-04-08 09:33:00-04:00      -0.590  
                222115  2024-04-08 09:34:00-04:00      -1.010  
...                                           ...         ...  
ZTS  2024-04-19 1108850 2024-04-19 15:56:00-04:00       0.000  
                1108851 2024-04-19 15:57:00-04:00       0.000  
                1108852 2024-04-19 15:58:00-04:00       0.015  
                1108853 2024-04-19 15:59:00-04:00      -0.095  
                1108854 2024-04-19 16:00:00-04:00      -0.105  

[1702619 rows x 7 columns]

In [207]:
def getgain(target):
    df = df_with_diff.loc[target].copy()
    for date,day in df.groupby('date'):
        missed = day['high'] < 0.02 + day['open']
        eodgain = day['close'].iloc[-1] - day['open']
        df.loc[date,'eodgain'] = eodgain
        df.loc[date,'missed']= missed
        df.loc[date,'eodmiss'] = missed*eodgain
    return df

In [229]:
df = getgain("MSFT")
loss_miss = (df['eodgain'] * df['missed']).sum()
gain = 0.02 * (1- df['missed'].sum())

miss = df['missed'].mean()

print(gain+loss_miss)

-570.4325


In [230]:
df.groupby('date')['eodgain'].sum()

date
2024-04-08    -160.9297
2024-04-09     855.3855
2024-04-10     711.6668
2024-04-11     604.1067
2024-04-12       8.6027
2024-04-15   -2361.5512
2024-04-16    -605.4223
2024-04-17    -623.1416
2024-04-18   -1645.2853
2024-04-19    -546.3844
Name: eodgain, dtype: float64

In [235]:
df.groupby('date')['eodgain'].sum().sum()

-3762.952800000002

In [237]:
df['close'].iloc[-30:-1].mean()  - df['close'].iloc[0:30].mean()

-25.68594931034471

In [238]:
df['eodgain'].mean()

-0.9623920204603583